- title: "Immuno-oncology workflow"
- author: "Margaret Paiva"
- date: "3/19/2022"
- output: R notebook

## Dependencies

In [1]:
#install.packages('IRkernel')
#IRkernel::installspec()

In [2]:
# cran packages
x <- c(
    'dplyr',
    'tidyr', 
    'ggplot2',
    'miceadds' # to source all files at once
)
# bioconductor packages
y <- c(
    'Seurat', 
    'nichenetr'
)

In [3]:
# install cran packages
for (pkg in x) {
    if (!pkg %in% rownames(installed.packages())) {
        print(pkg)
        install.packages(pkg)
    }
}

[1] "miceadds"


also installing the dependencies ‘mice’, ‘mitools’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [4]:
# install bioconductor packages
if (!requireNamespace("BiocManager", quietly=TRUE))
    install.packages("BiocManager")
for (pkg in y) {
    if (!requireNamespace(pkg, quietly=TRUE)) {
        print(pkg)
        BiocManager::install(pkg)
    }
}

In [4]:
# load packages
load_lib <- function(x) {
    suppressPackageStartupMessages(library(x, character.only = TRUE))
}
invisible(lapply(c(x, y), load_lib))

In [5]:
source.all('./nichenetr/R')

## Data

In [10]:
# ligand_target_matrix  <- readRDS(
#     url("https://zenodo.org/record/3260758/files/ligand_target_matrix.rds"))
ligand_target_matrix  <- readRDS("data/ligand_target_matrix.rds")
ligand_target_matrix[1:5,1:5]

,CXCL1,CXCL2,CXCL3,CXCL5,PPBP
A1BG,3.534343e-04,4.041324e-04,3.729920e-04,3.080640e-04,2.628388e-04
A1BG-AS1,1.650894e-04,1.509213e-04,1.583594e-04,1.317253e-04,1.231819e-04
A1CF,5.787175e-04,4.596295e-04,3.895907e-04,3.293275e-04,3.211944e-04
A2M,6.027058e-04,5.996617e-04,5.164365e-04,4.517236e-04,4.590521e-04
A2M-AS1,8.898724e-05,8.243341e-05,7.484018e-05,4.912514e-05,5.120439e-05


In [9]:
# lr_network = readRDS(
#     url("https://zenodo.org/record/3260758/files/lr_network.rds"))
lr_network = readRDS("data/lr_network.rds")
head(lr_network)


from,to,source,database
<chr>,<chr>,<chr>,<chr>
CXCL1,CXCR2,kegg_cytokines,kegg
CXCL2,CXCR2,kegg_cytokines,kegg
CXCL3,CXCR2,kegg_cytokines,kegg
CXCL5,CXCR2,kegg_cytokines,kegg
PPBP,CXCR2,kegg_cytokines,kegg
CXCL6,CXCR2,kegg_cytokines,kegg


In [8]:
# weighted_networks = readRDS(
#     url("https://zenodo.org/record/3260758/files/weighted_networks.rds"))
weighted_networks = readRDS("data/weighted_networks.rds")
weighted_networks_lr = weighted_networks$lr_sig %>% 
    inner_join(lr_network %>% distinct(from,to), 
               by = c("from","to"))
# interactions and their weights in the ligand-receptor + signaling network
head(weighted_networks$lr_sig) 
# interactions and their weights in the gene regulatory network
head(weighted_networks$gr) 

from,to,weight
<chr>,<chr>,<dbl>
A1BG,ABCC6,0.42164389
A1BG,ACE2,0.10074109
A1BG,ADAM10,0.09698978
A1BG,AGO1,0.05245879
A1BG,AKT1,0.08553468
A1BG,ANXA7,0.45659947


from,to,weight
<chr>,<chr>,<dbl>
A1BG,A2M,0.02944793
AAAS,GFAP,0.02904173
AADAC,CYP3A4,0.04215706
AADAC,IRF8,0.02745930
AATF,ATM,0.03299318
AATF,ATR,0.03545127


In [ ]:
ligands = lr_network %>% pull(from) %>% unique()
receptors = lr_network %>% pull(to) %>% unique()